# 6050 Project 



Link to access GloVe files can be found here:  http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
# import stuff
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0-preview is required
import tensorflow as tf
#from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import keras
import pickle
import random

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Train Data
path = 'drive/MyDrive/DS_6050_Project/train.csv'
df = pd.read_csv(path)

In [ ]:
#TestData
path2 = 'drive/MyDrive/DS_6050_Project/test_with_labels.xlsx'
df2 = pd.read_excel(path2)

In [ ]:
df3 = pd.concat([df, df2])

In [ ]:
#Number of Null Values that need to be deleted
len(df[df['text'].isnull()])

39

All of the values above were classified as fake news so want to make sure deleting these won't cause a problem.

In [ ]:
#Looking at the distribution of data
print(len(df[df['label']==1]))
print(len(df[df['label']==0]))

10413
10387


I don't think losing these 39 rows will be a problem so I am going to drop them.

In [ ]:
# #Removing the null values for train
# df = df[df['text'].notnull()]

In [ ]:
# #Removing the null values for test
# df2 = df2[df2['text'].notnull()]

In [ ]:
# #Removing the null values for test
df3 = df3[df3['text'].notnull()]

In [ ]:
# pull out the text and label, shuffles it and puts it in a list
def load_kagglefakenews(df):
    #load training data and put into arrays
    train_data = df['text'].values.tolist() #'text' column contains articles
    train_labels = df['label'].values.tolist() #'label' column contains labels

    #Randomly shuffle data and labels together
    combo = list(zip(train_data, train_labels))
    random.shuffle(combo)
    train_data, train_labels = zip(*combo)
    del df #clear up memory

    return np.asarray(train_data).tolist(), np.asarray(train_labels).tolist()

In [ ]:
# train_data, train_labels = load_kagglefakenews(df)

In [ ]:
# test_data, test_labels = load_kagglefakenews(df2)

In [ ]:
train_data, train_labels = load_kagglefakenews(df3)

In [ ]:
MAX_NB_WORDS=50000 #dictionary size
MAX_SEQUENCE_LENGTH=1500 #max word length of each individual article
EMBEDDING_DIM=300 #dimensionality of the embedding vector (50, 100, 200, 300)
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')

def tokenize_trainingdata(texts, labels):
    tokenizer.fit_on_texts(texts)
    pickle.dump(tokenizer, open('tokenizer.p', 'wb'))

    sequences = tokenizer.texts_to_sequences(texts)

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    data = tf.keras.utils.pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

    labels = tf.keras.utils.to_categorical(labels, num_classes=len(set(labels)))

    return data, labels, word_index

#and run it
X, Y, word_index = tokenize_trainingdata(train_data, train_labels)
#A, B, word_index2 = tokenize_trainingdata(test_data, test_labels)

Found 283796 unique tokens.


In [ ]:
# #Use all of the train data
# train_data = X
# train_labels = Y

In [ ]:
#Use all of the test data
# test_data = A
# test_labels = B

In [ ]:
train_data = X[:int(len(X)*0.70)]
train_labels = Y[:int(len(X)*0.70)]
test_data = X[int(len(X)*0.7):int(len(X)*0.90)]
test_labels = Y[int(len(X)*0.7):int(len(X)*0.90)]
valid_data = X[int(len(X)*0.90):]
valid_labels = Y[int(len(X)*0.90):]

In [ ]:
# train_data = X[:int(len(X)*0.90)]
# train_labels = Y[:int(len(X)*0.90)]
# # test_data = X[int(len(X)*0.9):int(len(X)*0.95)]
# # test_labels = Y[int(len(X)*0.9):int(len(X)*0.95)]
# valid_data = X[int(len(X)*0.90):]
# valid_labels = Y[int(len(X)*0.90):]

In [ ]:
def load_embeddings(word_index, embeddingsfile='drive/MyDrive/DS_6050_Project/glove.6B.%id.txt' %EMBEDDING_DIM):
    embeddings_index = {}
    f = open(embeddingsfile, 'r', encoding='utf8')
    for line in f:
        #here we parse the data from the file
        values = line.split(' ') #split the line by spaces
        word = values[0] #each line starts with the word
        coefs = np.asarray(values[1:], dtype='float32') #the rest of the line is the vector
        embeddings_index[word] = coefs #put into embedding dictionary
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))

    embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    
    return embedding_matrix
    
#and build the embedding matrix
embedding_matrix = load_embeddings(word_index)

Found 400000 word vectors.


In [ ]:
embedding_matrix.shape

(250671, 300)

In [ ]:
model = keras.Sequential([

     keras.layers.Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix], input_length = MAX_SEQUENCE_LENGTH, trainable=False),
     keras.layers.Conv1D(64, 5, activation='relu'),
     keras.layers.MaxPool1D(5),
     keras.layers.Conv1D(128, 3, activation='relu'), 
     keras.layers.MaxPool1D(5),
     keras.layers.Conv1D(256, 2, activation='relu'),
     keras.layers.GlobalAveragePooling1D(),
     keras.layers.Dense(2048, activation='relu'),
     keras.layers.Dropout(0.5),
     keras.layers.Dense(512, activation='relu'),
     keras.layers.Dropout(0.5),
     keras.layers.Dense(2, activation='softmax')
    #  keras.layers.Dense(1, activation='sigmoid')
     ])


In [ ]:
model.build(input_shape = embedding_matrix.shape[1])

In [ ]:
# values = []
# for v in test_labels:
#   values.append(v[1])


In [ ]:
# print(values)

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1500, 300)         85139100  
                                                                 
 conv1d_12 (Conv1D)          (None, 1496, 64)          96064     
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 299, 64)          0         
 1D)                                                             
                                                                 
 conv1d_13 (Conv1D)          (None, 297, 128)          24704     
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 59, 128)          0         
 1D)                                                             
                                                                 
 conv1d_14 (Conv1D)          (None, 58, 256)          

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

In [ ]:
model.fit(train_data, train_labels,
          validation_data=(valid_data, valid_labels),
          epochs=25, batch_size=64)

Epoch 1/25
284/284 [==============================] - 7s 21ms/step - loss: 0.3803 - accuracy: 0.8264 - val_loss: 0.2919 - val_accuracy: 0.8852
Epoch 2/25
284/284 [==============================] - 5s 19ms/step - loss: 0.2606 - accuracy: 0.8998 - val_loss: 0.2547 - val_accuracy: 0.9049
Epoch 3/25
284/284 [==============================] - 6s 19ms/step - loss: 0.2308 - accuracy: 0.9143 - val_loss: 0.2616 - val_accuracy: 0.9010
Epoch 4/25
284/284 [==============================] - 6s 19ms/step - loss: 0.2009 - accuracy: 0.9278 - val_loss: 0.3232 - val_accuracy: 0.8817
Epoch 5/25
284/284 [==============================] - 5s 19ms/step - loss: 0.1781 - accuracy: 0.9359 - val_loss: 0.2985 - val_accuracy: 0.8948
Epoch 6/25
284/284 [==============================] - 5s 19ms/step - loss: 0.1490 - accuracy: 0.9452 - val_loss: 0.3400 - val_accuracy: 0.8841
Epoch 7/25
284/284 [==============================] - 5s 19ms/step - loss: 0.1269 - accuracy: 0.9534 - val_loss: 0.3685 - val_accuracy: 0.8760

In [ ]:
model.evaluate(test_data, test_labels)

163/163 [==============================] - 1s 5ms/step - loss: 0.9030 - accuracy: 0.8576


[0.9029831290245056, 0.8576107621192932]

**TRYING TO USE BERT**

In [ ]:
######### Trying something new https://towardsdatascience.com/a-beginners-guide-to-use-bert-for-the-first-time-2e99b8c5423

In [ ]:
# !pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.2 MB/s 
     |████████████████████████████████| 7.6 MB 46.0 MB/s 
     |████████████████████████████████| 163 kB 59.2 MB/s 


In [ ]:
from transformers import DistilBertTokenizerFast, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
#Resetting the train_data so they are no longer GloVe Encodings
train_data, train_labels = load_kagglefakenews(df3)

In [ ]:
train_data_2 = train_data[:int(len(train_data)*0.7)]
train_labels_2 = train_labels[:int(len(train_labels)*0.7)]
test_data = train_data[int(len(train_data)*0.7):int(len(train_data)*0.9)]
test_labels = train_labels[int(len(train_labels)*0.7):int(len(train_labels)*0.9)]
valid_data = train_data[int(len(train_data)*0.9):]
valid_labels = train_labels[int(len(train_labels)*0.9):]

In [ ]:
train_encodings = tokenizer(train_data_2, truncation=True, padding=True)

In [ ]:
val_encodings = tokenizer(valid_data, truncation=True, padding=True)
test_encodings = tokenizer(test_data, truncation=True, padding=True)

In [ ]:
import torch

In [ ]:
class FakeNewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = FakeNewsDataset(train_encodings, train_labels_2)
val_dataset = FakeNewsDataset(val_encodings, valid_labels)
test_dataset = FakeNewsDataset(test_encodings, test_labels)

In [ ]:
####### Trying new method using https://developer.habana.ai/tutorials/pytorch/distilbert-sequence-classification-with-imdb-reviews/

In [ ]:
def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="test-2",
    #evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    #load_best_model_at_end=True,
    #metric_for_best_model="accuracy"
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

Step,Training Loss
500,0.338700
1000,0.263100


Saving model checkpoint to test-2/checkpoint-500
Configuration saved in test-2/checkpoint-500/config.json
Model weights saved in test-2/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test-2/checkpoint-1000
Configuration saved in test-2/checkpoint-1000/config.json
Model weights saved in test-2/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1136, training_loss=0.29337386346199146, metrics={'train_runtime': 492.4006, 'train_samples_per_second': 36.891, 'train_steps_per_second': 2.307, 'total_flos': 2406270296586240.0, 'train_loss': 0.29337386346199146, 'epoch': 1.0})

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.1 MB/s 
     |████████████████████████████████| 95 kB 4.9 MB/s 
     |████████████████████████████████| 115 kB 38.6 MB/s 
     |████████████████████████████████| 212 kB 43.5 MB/s 
     |████████████████████████████████| 127 kB 58.2 MB/s 
     |████████████████████████████████| 115 kB 55.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [ ]:
from datasets import load_metric
metric = load_metric("accuracy")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


In [ ]:
#Validation Accuracy
metrics = trainer.evaluate()
metrics["eval_samples"] = len(valid_data)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)


***** Running Evaluation *****
  Num examples = 2596
  Batch size = 16


***** eval metrics *****
  epoch                   =        1.0
  eval_accuracy           =     0.9237
  eval_loss               =     0.2413
  eval_runtime            = 0:00:23.22
  eval_samples            =       2596
  eval_samples_per_second =    111.787
  eval_steps_per_second   =      7.019


In [ ]:
#Test Accuracy
metrics = trainer.evaluate(test_dataset)
metrics["eval_samples"] = len(test_dataset)
trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)

***** Running Evaluation *****
  Num examples = 5190
  Batch size = 16


***** test metrics *****
  epoch                   =        1.0
  eval_accuracy           =     0.9162
  eval_loss               =     0.2535
  eval_runtime            = 0:00:44.98
  eval_samples            =       5190
  eval_samples_per_second =    115.365
  eval_steps_per_second   =      7.224


#**RNN MODEL**

In [ ]:
df = df[df['title'].notnull()]
df_test = df_test[df_test['title'].notnull()]

In [ ]:
df= df.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
import re
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

corpus = []
for i in range(0, len(df['title'])):
    titles = df['title'][i]
    titles = titles.lower()
    titles = titles.split()
    
    titles = [stemmer.stem(word) for word in titles if not word in stopwords.words('english')]
    titles = ' '.join(titles)
    corpus.append(titles)

In [ ]:
test_corpus = []
for i in range(0, len(df_test['title'])):
    test_title = df_test['title'][i]
    test_title = test_title.lower()
    test_title = test_title.split()
    
    test_title = [stemmer.stem(word) for word in test_title if not word in stopwords.words('english')]
    test_title = ' '.join(test_title)
    test_corpus.append(test_title)

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
voc_size = 500
onehot_corpus = [one_hot(words, voc_size) for words in corpus]

In [ ]:
onehot_test = [one_hot(words, voc_size) for words in test_corpus]

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length = 20

In [ ]:
X_final=np.array(pad_sequences(onehot_corpus, padding="post", maxlen=sent_length))
y_final=np.array(df['label'])

In [ ]:
X_final_test=np.array(pad_sequences(onehot_test, padding="post", maxlen=sent_length))
y_final_test=np.array(df_test['label'])

In [ ]:
X_final = X_final[:int(len(X_final)*0.95)]
y_final = y_final[:int(len(y_final)*0.95)]
x_valid = X_final[:int(len(X_final)*0.95)]
y_valid = y_final[:int(len(y_final)*0.95)]

In [ ]:
import keras
#input length is the number of words in the titles
#500 is the vocab size

embedding_vector_features = 400
model = keras.Sequential([ 
    keras.layers.Embedding(500,embedding_vector_features,input_length=20),
    keras.layers.Dropout(0.4),
    keras.layers.LSTM(100),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 400)           200000    
                                                                 
 dropout (Dropout)           (None, 20, 400)           0         
                                                                 
 lstm (LSTM)                 (None, 100)               200400    
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 400,501
Trainable params: 400,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_final,y_final,validation_data=(x_valid,y_valid),epochs=40,batch_size=64)

Epoch 1/40
300/300 [==============================] - 5s 9ms/step - loss: 0.2882 - accuracy: 0.8787 - val_loss: 0.2139 - val_accuracy: 0.9108
Epoch 2/40
300/300 [==============================] - 3s 9ms/step - loss: 0.2201 - accuracy: 0.9121 - val_loss: 0.1880 - val_accuracy: 0.9256
Epoch 3/40
300/300 [==============================] - 2s 8ms/step - loss: 0.2016 - accuracy: 0.9189 - val_loss: 0.1701 - val_accuracy: 0.9313
Epoch 4/40
300/300 [==============================] - 2s 8ms/step - loss: 0.1904 - accuracy: 0.9221 - val_loss: 0.1644 - val_accuracy: 0.9332
Epoch 5/40
300/300 [==============================] - 2s 8ms/step - loss: 0.1798 - accuracy: 0.9280 - val_loss: 0.1875 - val_accuracy: 0.9358
Epoch 6/40
300/300 [==============================] - 2s 8ms/step - loss: 0.1693 - accuracy: 0.9311 - val_loss: 0.1496 - val_accuracy: 0.9351
Epoch 7/40
300/300 [==============================] - 2s 8ms/step - loss: 0.1573 - accuracy: 0.9369 - val_loss: 0.1375 - val_accuracy: 0.9473
Epoch 

In [ ]:
model.evaluate(X_final_test,y_final_test)

159/159 [==============================] - 0s 3ms/step - loss: 3.6327 - accuracy: 0.6298


[3.6327438354492188, 0.6298342347145081]